### レース結果データをスクレイピング ###

#### 1-1 開催日データをスクレイピング ####

In [1]:
"""2024年の開催日データを取得"""

# ライブラリをインポート
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

# 年と月をループで回す
year = 2024 # 2024年
dates = [] # 開催日のリストを定義

for month in range(1, 13):  # 1月〜12月
    time.sleep(2) # サーバー負荷軽減
    url = f'https://race.netkeiba.com/top/calendar.html?year={year}&month={month}'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    # aタグのhref属性からkaisai_dateの値を抽出
    a_all = soup.find_all("a", href=True) #aタグを全て抜き出してfor文で１つずつ抜き出す
    for a in a_all:
        match = re.search(r"kaisai_date=(\d{8})", a["href"]) #正規表現で抜き出し
        if match:
            dates.append(match.group(1)) #数字の部分だけを抜き出す

print(dates[:10])

['20240106', '20240107', '20240108', '20240113', '20240114', '20240120', '20240121', '20240127', '20240128', '20240203']


#### 1-2 レースIDをスクレイピング ####

In [5]:
""" レースIDを取得 """

# 必要なライブラリをインポート
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
from bs4 import BeautifulSoup
import time
import re

# Chromeの設定
chrome_options = Options()
chrome_options.add_argument('--headless') #画面を表示させない

# ドライバ初期化
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                      options=chrome_options)

race_ids = [] # race_idを入れるリスト
for date in tqdm(dates): # 開催日のリストを１つずつ取り出してurlに接続
  time.sleep(2)
  url = f'https://race.netkeiba.com/top/race_list.html?kaisai_date={date}'
  driver.get(url)

  # ページのHTMLを取得
  html = driver.page_source
  soup = BeautifulSoup(html, "html.parser")

  # aタグのhref属性からrace_idを抜き出す
  a_all = soup.find_all("a", href=True)
  for a in a_all:
    match = re.search(r"race_id=(\d{12})", a["href"])
    if match:
      race_ids.append(match.group(1))

driver.quit()

# テキストファイルに保存
with open('race_ids.txt', 'w') as file:
    for race_id in race_ids:
        file.write(f"{race_id}\n")

# 最初の10個のデータを出力
print(race_ids[:10])

100%|██████████| 106/106 [06:02<00:00,  3.42s/it]


['202406010101', '202406010101', '202406010102', '202406010102', '202406010103', '202406010103', '202406010104', '202406010104', '202406010105', '202406010105']
